## Libraries + API Keys + Parameters

In [ ]:
!pip install crewai
!pip install crewai-tools
!pip install openai

  Using cached crewai_tools-0.33.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached embedchain-0.1.126-py3-none-any.whl.metadata (9.3 kB)
  Using cached lancedb-0.18.0-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (4.0 kB)
  Using cached pyright-1.1.393-py3-none-any.whl.metadata (6.6 kB)
  Using cached pytube-15.0.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached alembic-1.14.1-py3-none-any.whl.metadata (7.4 kB)
  Using cached chromadb-0.5.23-py3-none-any.whl.metadata (6.8 kB)
  Using cached cohere-5.13.11-py3-none-any.whl.metadata (3.4 kB)
  Using cached gptcache-0.1.44-py3-none-any.whl.metadata (24 kB)
  Using cached langchain_cohere-0.3.5-py3-none-any.whl.metadata (6.7 kB)
  Using cached langchain_community-0.3.16-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_openai-0.2.14-py3-none-any.whl.metadata (2.7 kB)
  Using cached langsmith-0.1.147-py3-none-any.whl.metadata (14 kB)
  Using cached mem0ai-0.1.48-py3

In [ ]:
# Retrieve the API keys securely from Google Colab's user data
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
import os
# Set the API keys as environment variables for accessibility
os.environ['OPENAI_API_KEY'] = openai_api_key

In [ ]:
from crewai_tools import DirectoryReadTool, PDFSearchTool, FileWriterTool
from crewai import Agent, Crew, Process, Task
from crewai.tasks.task_output import TaskOutput

from langchain_openai import ChatOpenAI
from IPython.display import Markdown
import pandas as pd

In [ ]:
inputs = {
    'folder': 'invoices',
    'requirements': 'all invoices from Simonis and Braun',
    'columns': ['Date', "Description", 'Total Amount', 'Issuer', "Receiver"],
    'question' : 'what is the total amount invoiced and all the names of personnel involved in the latest invoive file'
}

## Assistant Agent - retrieve necessary documents

In [ ]:
assistant_agent = Agent(
    role="Invoice Retrieval Assistant",
    goal=f"""
        Your task is to search the files within the folder **{inputs['folder']}** and locate the correct document based on the following criteria:
        **{inputs['requirements']}**
        Once identified, extract and retrieve the relevant data from the document.
    """,
    backstory="""
        You are an efficient and detail-oriented assistant specializing in organizing and retrieving relevant data from documents.
    """,
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0.8),  # Using GPT-4 model
    tools=[DirectoryReadTool(), PDFSearchTool()]
)

In [ ]:
assistant_task = Task(
    description=f"""
        Search for the correct file within **{inputs['folder']}** that matches the following criteria:
        **{inputs['requirements']}**
        Once found, extract all the data from the relevant file.
    """,
    expected_output="""
        The extracted data from the relevant file, formatted for clarity and completeness.
    """,
    agent=assistant_agent
)


## Organizer Agent - Format into CSV

In [ ]:
organizer_agent = Agent(
    role="Data Organization Assistant",
    goal=f"""
        Process the provided data and format it as a CSV file with the specified columns:
        **{inputs['columns']}**
        Save the results strictly in CSV format without any additional text or formatting.
    """,
    backstory="""
        You are an efficient and detail-oriented assistant specializing in structuring and organizing data into CSV format.
    """,
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0.8),  # Using GPT-4 model
)


In [ ]:
organizer_task = Task(
    description=f"""
        Format the provided data into a CSV file with the specified columns:
        **{inputs['columns']}**
        Ensure that all numeric values use a decimal point (.) instead of a comma (,).
        Data should be formatted as DD-MM-YYYY.
        Save the results strictly in CSV format without any additional text.
    """,
    expected_output="""
        A properly formatted CSV file containing the correct values in the specified columns.
    """,
    agent=organizer_agent,
)


## Analyst Agent - Answer query

In [ ]:
analyst_agent = Agent(
    role="Data Analyst Assistant",
    goal=f"""
        Analyze the provided data and accurately answer the following question:
        **{inputs['question']}**
        Base your response strictly on data and factual analysis.
    """,
    backstory="""
        You are a detail-oriented data analyst, skilled at extracting insights and answering questions based on facts and data.
    """,
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0.8),  # Using GPT-4 model
)


In [ ]:
analyst_task = Task(
    description=f"""
        Analyze the provided data and answer the following question:
        **{inputs['question']}**
        Ensure your response is based strictly on data-driven insights.
    """,
    expected_output="""
        A correct, data-backed answer along with the name of the file used for analysis.
    """,
    agent=analyst_agent,
)


## Assemble crew

In [ ]:
crew = Crew(
    agents=[assistant_agent, organizer_agent, analyst_agent],  # List of agents involved
    tasks=[assistant_task, organizer_task, analyst_task],  # List of tasks to execute
    verbose=True,
    process=Process.sequential  # Ensuring tasks are processed in order
)
result = crew.kickoff()  # Start the task execution

# Agent: Invoice Retrieval Assistant
## Task: 
        Search for the correct file within **invoices** that matches the following criteria:  
        **all invoices from Simonis and Braun**  
        Once found, extract all the data from the relevant file.
    


# Agent: Invoice Retrieval Assistant
## Using tool: List files in directory
## Tool Input: 
"{\"directory\": \"invoices\"}"
## Tool Output: 
File paths: 
-invoices/Hammes_and_sons_2024-12.pdf
- invoices/Torphy_2025-02.pdf
- invoices/Braun_2022-10.pdf
- invoices/simonis_2025-03.pdf
- invoices/Gulgowski_and_sons_2025-02.pdf


# Agent: Invoice Retrieval Assistant
## Using tool: Search a PDF's content
## Tool Input: 
"{\"query\": \"Simonis and Braun\", \"pdf\": \"invoices/Braun_2022-10.pdf\"}"
## Tool Output: 
Relevant Content:
Sample - Do not pay! Braun Ltd Treutel Unions 505 38003-6849 South Tobyside Date: 2022-10-04 Invoice No.: 31500 Invoice Dear Mr. Cormier, We hereby invoice the following deliveries and services. Pos. Amount

In [ ]:
# raw data from the invoices
Markdown(result.tasks_output[0].raw)

**Invoice from Braun Ltd (Braun_2022-10.pdf):**
```
Sample - Do not pay! 
Braun Ltd 
Treutel Unions 505 
38003-6849 South Tobyside 

Date: 2022-10-04 
Invoice No.: 31500 

Dear Mr. Cormier, 
We hereby invoice the following deliveries and services.

Pos. Amount Unit Description Price Total 
1 73 Pcs. Quis ut excepturi quia odit.474,92 € 34.669,16 € 
2 79 Pcs. Et qui voluptatem libero.554,69 € 43.820,51 € 
3 92 Pcs. Accusamus ea autem. 830,53 € 76.408,76 € 
4 67 Pcs. Consectetur ratione explicabo.196,93 € 13.194,31 € 

Price net 168.092,74 € 
plus 10% VAT 16.809,27 € 
Invoice total 184.902,01 € 

Payment details: Payment within 90 days 
Braun Ltd - Treutel Unions 505 - 38003-6849 South Tobyside 

Stehr-Heaney Donato Cormier Leopold Forks 711 24085 McCluremouth Saint Pierre and Miquelon 

WARNING! This document is only for testing purposes. Do not make any payment! - Generated by invoicefaker.com 
Braun Ltd Treutel Unions 505 38003-6849 South Tobyside 
CEO: Liliana Jast EN715009365 586 / 5 / 535 
Bank details: NL07LBKP8434276625 KPKMGASKW1P
```

**Invoice from Simonis LLC (simonis_2025-03.pdf):**
```
Sample - Do not pay! 
INVOICE 84951 
DATE 05.02.2025 
PAYMENT DETAILS 07.03.2025 

DESCRIPTION PRICE AMOUNT TOTAL 
Deleniti consequatur hic omnis. 918,80 € 59 54.209,20 € 
Quis qui cum eligendi. 162,67 € 82 13.338,94 € 
Aut error voluptatem voluptatem. 976,13 € 82 80.042,66 € 
Assumenda nemo. 171,00 € 29 4.959,00 € 
Qui ab. 309,57 € 76 23.527,32 € 

PRICE NET 176.077,12 € 
plus 7% VAT 12.325,40 € 
INVOICE TOTAL 188.402,52 € 

CUSTOMER Norwood Spinka 
ADDRESS Aurelia Wall 784 
CITY 42644-0051 South Maudieland 

Simonis LLC 
COMPANY Barney Village 891 
ADDRESS 24163-6226 Kaseytown CITY 

PAYMENT DETAILS: Payment within 30 days 
WARNING! This document is only for testing purposes. Do not make any payment! - Generated by invoicefaker.com 

Simonis LLC Barney Village 891 24163-6226 Kaseytown CEO: Garry Kilback EN658568060 193 / 590 / 193 
Bank details: RS97378587727026913595 AADDWABV
```
```

In [ ]:
# table structured
print(result.tasks_output[1].raw)

```
Date,Description,Total Amount,Issuer,Receiver
04-10-2022,Quis ut excepturi quia odit.,34669.16,Braun Ltd,Mr. Cormier
04-10-2022,Et qui voluptatem libero.,43820.51,Braun Ltd,Mr. Cormier
04-10-2022,Accusamus ea autem.,76408.76,Braun Ltd,Mr. Cormier
04-10-2022,Consectetur ratione explicabo.,13194.31,Braun Ltd,Mr. Cormier
04-10-2022,Invoice Total,184902.01,Braun Ltd,Mr. Cormier
05-02-2025,Deleniti consequatur hic omnis.,54209.20,Simonis LLC,Norwood Spinka
05-02-2025,Quis qui cum eligendi.,13338.94,Simonis LLC,Norwood Spinka
05-02-2025,Aut error voluptatem voluptatem.,80042.66,Simonis LLC,Norwood Spinka
05-02-2025,Assumenda nemo.,4959.00,Simonis LLC,Norwood Spinka
05-02-2025,Qui ab.,23527.32,Simonis LLC,Norwood Spinka
05-02-2025,Invoice Total,188402.52,Simonis LLC,Norwood Spinka
```


In [ ]:
# write to csv
from io import StringIO

df = pd.read_csv(StringIO(result.tasks_output[1].raw))
df.dropna(inplace=True)
df.to_csv('result.csv')

In [ ]:
# answer to the question
Markdown(result.tasks_output[2].raw)

The total amount invoiced including VAT for the latest invoice file dated 05-02-2025 is 188,402.52. The personnel involved in this invoice are Simonis LLC as the issuer and Norwood Spinka as the receiver. This data is from the invoice file "05-02-2025."